Prototype of GERICS preprocessing chain for the LUCAS 3D wind analysis. 

Challenges:
- shifted grids in vertical and horizontal dimensions

In [1]:
import xarray as xr
import numpy as np

In [2]:
# open a single dataset as example
ds = xr.open_dataset("../data/GERICS/062008/e062008t201512_wnd.nc")

In [3]:
# Collapse time dimension to make data smaller. Doesn't matter here as we focus on cleaning up rlat, rlon and lev
ds = ds.isel({"time":1})

### Looking at the data

In [4]:
ds

<xarray.Dataset>
Dimensions:       (rlon: 129, rlat: 121, rlon_2: 129, rlat_2: 121, lev: 27,
                   nhyi: 28, nhym: 27, lev_2: 27)
Coordinates:
    time          float64 2.015e+07
  * rlon          (rlon) float64 -31.73 -31.29 -30.85 ... 23.71 24.15 24.59
  * rlat          (rlat) float64 -26.73 -26.29 -25.85 ... 25.19 25.63 26.07
  * rlon_2        (rlon_2) float64 -31.51 -31.07 -30.63 ... 23.93 24.37 24.81
  * rlat_2        (rlat_2) float64 -26.51 -26.07 -25.63 ... 25.41 25.85 26.29
  * lev           (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 23.0 24.0 25.0 26.0 27.0
  * lev_2         (lev_2) float64 1.0 2.0 3.0 4.0 5.0 ... 24.0 25.0 26.0 27.0
Dimensions without coordinates: nhyi, nhym
Data variables:
    rotated_pole  int32 ...
    hyai          (nhyi) float64 ...
    hybi          (nhyi) float64 ...
    hyam          (nhym) float64 ...
    hybm          (nhym) float64 ...
    FIB           (rlat, rlon) float32 ...
    T             (lev, rlat, rlon) float32 ...
    U             (lev, rlat, rlon_2) float32 ...
    V             (lev, rlat_2, rlon) float32 ...
    PS            (rlat, rlon) float32 ...
    FI            (lev_2, rlat, rlon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    source:       REMO
    institution:  Max Planck Institute for Meteorology
    history:      Thu May 12 09:40:56 2022: cdo selcode,129,130,131,132,134,1...
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

In [5]:
ds["lev_2"]

<xarray.DataArray 'lev_2' (lev_2: 27)>
array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.])
Coordinates:
    time     float64 2.015e+07
  * lev_2    (lev_2) float64 1.0 2.0 3.0 4.0 5.0 ... 23.0 24.0 25.0 26.0 27.0
Attributes:
    standard_name:  hybrid_sigma_pressure
    long_name:      hybrid level at layer midpoints
    formula:        hyam hybm (mlev=hyam+hybm*aps)
    formula_terms:  ap: hyam b: hybm ps: aps
    units:          level
    positive:       down

In [6]:
ds["lev"]

<xarray.DataArray 'lev' (lev: 27)>
array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.])
Coordinates:
    time     float64 2.015e+07
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 23.0 24.0 25.0 26.0 27.0
Attributes:
    standard_name:  hybrid_sigma_pressure
    long_name:      hybrid level at layer midpoints
    formula:        hyam hybm (mlev=hyam+hybm*aps)
    formula_terms:  ap: hyam b: hybm ps: aps
    units:          level
    positive:       down

### Handling the horizontal grid

**Approach**: Compute wind speeds at the center of the grid cell (i.e., `rlat`, `rlon`) from the components at the edges. This leaves one row and column undefined. Fill them with nans.

In [7]:
(ds.rlon_2.values - ds.rlon.values)[0]

0.21999999999999886

In [8]:
(ds.rlat_2.values - ds.rlat.values)[0]

0.21999999999999886

In [9]:
# Manual computation
s = (((ds["U"].values[:,1:,:-1]+ds["U"].values[:,:-1,:-1])/2)**2 + ((ds["V"].values[:,:-1,1:]+ds["U"].values[:,:-1,:-1])/2)**2)**(1./2)

In [10]:
s.shape

(27, 120, 128)

In [11]:
ds["U"].shape

(27, 121, 129)

Need to fill one column and one row of nans at western and southern margin of the domain

In [12]:
s_full = np.zeros(ds["U"].shape)*np.nan  # create numpy array of desired size

In [13]:
s_full[:,:-1,1:] = s  # fill from 2nd column and up to pen-ultimate row with actual s data

In [ ]:
# create a Dataset with coordinates time, rlat, rlon, lev
ds_s = xr.Dataset(
    data_vars=dict(
        s=(["lev", "rlat", "rlon"], s_full)
    ),
    coords=dict(
        lev=ds.lev,
        rlat=ds.rlat,
        rlon=ds.rlon,
    ),
    attrs=dict(
        long_name="wind speed",
        unit="m/s",
        grid_mapping="rotated pole",
    ),
)


In [ ]:
ds_s

In [ ]:
ds = ds.drop(["U", "V", "hyai", "hybi", "hyam", "hybm"]).drop_dims(["rlat_2", "rlon_2"])  # cleaning up, dropping variables that are not needed 

In [ ]:
ds = xr.merge([ds, ds_s])  # adding wind speeds back in

### Handling the vertical grid

Wind speeds are provided at `lev` but geopotential is not. Geopotential is on `lev_2` which is shifted by half a cell.
According to Kevin, it is a meaningful approach to average over two values of `lev_2` to arrive at the geopotential at the height of wind speeds. This is implemented below. 

In [ ]:
# Illustration for a single location
ds_example = ds.isel({"rlon": 60, "rlat": 60})

In [ ]:
ds_example

In [ ]:
# creata an array containing all FI values and the surface FI value stored in FIB
FI_all = list(ds_example.FI.values)
FI_all.append(float(ds_example.FIB.values)) # adding surface geopotential
FI_all = np.array(FI_all)

In [ ]:
FI_interpolated = (FI_all[1:]+FI_all[:-1])/2

In [ ]:
FI_interpolated

In [ ]:
#create a dataset with lev dimension

ds_FI = xr.Dataset(
    data_vars=dict(
        FI=(["lev"], FI_interpolated)
    ),
    coords=dict(
        lev=ds.lev,
    ),
    attrs=dict(
        long_name="interpolated geopotential",
        unit="m",
        grid_mapping="rotated pole",
    ),
)


In [ ]:
ds_example = ds_example.drop(["FI"]).drop_dims(["lev_2"])

In [ ]:
ds_example = xr.merge([ds_example, ds_FI])

In [ ]:
ds_example